In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split


In [2]:
##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [3]:
# 학습용 데이터셋 불러오기
import pandas as pd
card_data = pd.read_csv('대분류.csv')
print('훈련용 리뷰 개수 :',len(card_data)) # 훈련용 리뷰 개수 출력
card_data.head()

훈련용 리뷰 개수 : 1000


,발화,대분류
0,상담직원과 빠른연결했슴좋겠어요,불만
1,안기다리고. 빨리상담되었음좋겠다,불만
2,너무 오래기다리고 상담이 늦어져요,불만
3,상담직원과 통화가 매우 힘들다,불만
4,상담받으려면오래기다리는게불편함,불만


In [4]:
# 데이터 다루기 
df_new = card_data.rename(columns={'발화': 'context','대분류': 'label'})
df_new['label'].unique()
df_new.head()

data1 = df_new[df_new['label'] == '불만']
data2 = df_new[df_new['label'] == '칭찬']
data3 = df_new[df_new['label'] == '중립']

new_data = data1.append([data2], sort=False)
# new_data = data1.append([data2, data3], sort=False)
new_data = pd.DataFrame(new_data)
new_data.head()

,context,label
0,상담직원과 빠른연결했슴좋겠어요,불만
1,안기다리고. 빨리상담되었음좋겠다,불만
2,너무 오래기다리고 상담이 늦어져요,불만
3,상담직원과 통화가 매우 힘들다,불만
4,상담받으려면오래기다리는게불편함,불만


In [5]:
# 라벨링 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(new_data['label'])
new_data['label'] = encoder.transform(new_data['label'])
new_data.head()

# 라벨링된 분류명 {0: '불만', 1: '칭찬'}
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
print(mapping)

{0: '불만', 1: '칭찬'}


In [6]:
# Train / Test set 분리 및 모델 input 형식 맞추기 
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

dataset_train = []
for column in range(len(train)):
    dataset_train.append([train.iloc[column]['context'], str(train.iloc[column]['label'])])

dataset_test = []
for column in range(len(test)):
    dataset_test.append([test.iloc[column]['context'], str(test.iloc[column]['label'])])


train shape is: 794
test shape is: 199


In [7]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [8]:
# 파라메타 세팅
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [10]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [12]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [13]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [14]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))



/home/khw/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/25 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.8136572241783142 train acc 0.53125
epoch 1 train acc 0.703173076923077


  0%|          | 0/25 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.4986802935600281 train acc 0.71875
epoch 2 train acc 0.9022115384615383


...

epoch 3 batch id 1 loss 0.2898683547973633 train acc 0.9375
epoch 3 train acc 0.9825


  0%|          | 0/25 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.22302575409412384 train acc 0.9375
epoch 4 train acc 0.99125


  0%|          | 0/25 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.01343621127307415 train acc 1.0
epoch 5 train acc 0.99875


In [15]:
# 테스트 문장 예측
test_sentence = '친절하게 말해주셔서 감사합니다.'
test_label = 1 # 실제 질병

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['발화내용', '분류']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids) # 모델 
    print(out)
    
    model.eval() # 평가 모드로 변경
    # 평가 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

# 모델 저장 
PATH = 'kobert_model.pth'

torch.save(model.state_dict(), PATH)

/home/khw/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[-2.3882,  2.8132]], device='cuda:0', grad_fn=<AddmmBackward>)


/home/khw/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 5 test acc 0.9553571428571429


In [16]:
# 모델 load 
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
PATH = 'kobert_model.pth'
model.load_state_dict(torch.load(PATH))


<All keys matched successfully>

In [17]:
# 테스트 함수 
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("불만")
                score = round(abs(np.sum(logits)) + 0.1,4)

            elif np.argmax(logits) == 1:
                test_eval.append("칭찬")
                score = round(abs(np.sum(logits)) + 0.1,4)

        if score > 0.9999:
            score = 0.99
        print(">> 입력하신 내용은 " + str(score) + '% ' + test_eval[0] + " 입니다.")


In [24]:
sentence = '친절 감사합니다'
predict(sentence)

>> 입력하신 내용은 0.1708% 칭찬 입니다.
